In [1]:
import pandas as pd
import numpy as np
import dhedfreader
import glob
import os
from mne.io import concatenate_raws, read_raw_edf
import codecs
from datetime import datetime, timedelta
import math
from scipy.signal import resample

In [2]:
DATA_DIR = '/share/wequ0318/CAP/physionet.org/files/capslpdb/1.0.0'
OUTPUT_DIR = '/share/wequ0318/cap_clean/cap'
channels = {'P4-O2': '_P4_O2', 'O2':'_O2', 'O2-A1': '_O2_A1', 'O2A1': '_O2_A1'}

In [3]:
ann2label = {
    "MCAP-A1": 0,
    "MCAP-A2": 1,
    "MCAP-A3": 2
}

In [4]:
def if_duplicate(time_duration_dict, epoch_time, duration):
    for t, d in time_duration_dict.items():
        t_t = datetime.strptime(t, "%H:%M:%S")
        e_t = datetime.strptime(epoch_time, "%H:%M:%S")
#         print(t, epoch_time)
        delta = e_t - t_t
        delta_sec = delta.total_seconds() + 86400 if delta.total_seconds() < -(12*3600) else delta.total_seconds()
        if int(delta_sec) < d: return True
        if delta_sec < 0: break
    return False

In [5]:
def extend_time(start_time, duration):
    start_time = start_time.replace('.', ':')
    s_t = datetime.strptime(start_time, "%H:%M:%S")
    ext_t = [(s_t + timedelta(seconds=d)).strftime("%H:%M:%S") for d in range(duration)]
    return np.array(ext_t)

In [6]:
max_duration_sec = 0
subject_id = 0
for f in glob.glob('{}/*.edf'.format(DATA_DIR)):
#     if 'n6.edf' not in f: continue
    subject_id += 1
    ann = f.replace('.edf', '.txt')
    df_ann = pd.read_csv(ann, skiprows=20 if 'n16.edf' in f else 21, delimiter='\t')
    raw = read_raw_edf(f, preload=True, stim_channel=None)
    
    sampling_rate = raw.info['sfreq']
    print("file {} contains channels {}".format(f, raw.info['ch_names']))
    select_ch = None
    suffix = None
    for ch in raw.info['ch_names']:
        if ch in list(channels.keys()):
            select_ch = ch
            suffix = channels[select_ch]
            break
    print("selected channel {} and suffix is {}".format(select_ch, suffix))
    out_f = os.path.join(OUTPUT_DIR, os.path.basename(f.replace('.edf', suffix+'.npz')))
#     if os.path.exists(out_f):
#         continue
    print('output to', out_f)  
    raw_ch_df = raw.to_data_frame(scaling_time=100.0)[select_ch]
    raw_ch_df = raw_ch_df.to_frame()
    raw_ch_df.set_index(np.arange(len(raw_ch_df)))

    # Get raw header
    file = codecs.open(f, 'r', encoding="utf-8")
    reader_raw = dhedfreader.BaseEDFReader(file)
    reader_raw.read_header()
    h_raw = reader_raw.header
    file.close()
    raw_start_dt = datetime.strptime(h_raw['date_time'], "%Y-%m-%d %H:%M:%S")
    df_ann['Event'].fillna('UNKNOWN', inplace=True)
    df_ann_cap = df_ann[df_ann['Event'].str.startswith('MCAP-')]
    # annotation start time
    start_time = h_raw['date_time'].split(' ')[0] + ' ' + list(df_ann_cap.head(1).to_dict()['Time [hh:mm:ss]'].values())[0]
    start_time = start_time.replace('.', ':')
    print('recording start time {}, annotation start time {}'.format(raw_start_dt, start_time))
    t1 = datetime.strptime(start_time, "%Y-%m-%d %H:%M:%S")
    # normally, annotation starts later than eeg.
    delta = t1 - raw_start_dt
    print(f"Time difference is {delta.total_seconds()} seconds")
    
    cnt = 0
    if delta.total_seconds() < 0 and delta.total_seconds() > -3000:
        print('handle the situation that annotation starts earlier than eeg')
        for _, row in df_ann_cap.iterrows():
            cnt += 1
            if row['Event'] not in list(ann2label.keys()):
                continue
            this_epoch_time = start_time.split(' ')[0] + ' ' + row['Time [hh:mm:ss]'].replace('.', ':')
            this_t = datetime.strptime(this_epoch_time, "%Y-%m-%d %H:%M:%S")
            delta = (this_t - raw_start_dt)
            print(this_t, raw_start_dt, delta, delta.total_seconds() > 0)
            if delta.total_seconds() > 0:
                start_time = this_epoch_time
                print('adjusted start time', start_time)
                break
    
    labels = []
    label_idx = []
    time_stamps = []
    last_epoch_time = start_time.split(' ')[-1]
    # start from where we have labels
    start_idx = delta.total_seconds() * sampling_rate if delta.total_seconds() >= 0 else (delta.total_seconds() + 86400) * sampling_rate

    duration_sec = 0
    
    time_duration = {}
    
    for _, row in df_ann_cap.iterrows():
        cnt -= 1
        if cnt > 0:
            continue
            
        if row['Event'] not in list(ann2label.keys()):
            continue

        this_duratioin_sec = int(row['Duration[s]']) if 'n16.edf' not in f else int(row['Duration [s]'])
        this_epoch_time = row['Time [hh:mm:ss]'].replace('.', ':')
        if if_duplicate(time_duration, this_epoch_time, this_duratioin_sec):
            continue
        else:
            time_duration[this_epoch_time] = this_duratioin_sec
            
        label = ann2label[row['Event']]
        last_t = datetime.strptime(last_epoch_time, "%H:%M:%S")
        this_t = datetime.strptime(this_epoch_time, "%H:%M:%S")
        
        # may skip some epochs
        offset = (this_t - last_t).total_seconds()
        print("cnt:", cnt, "last_epoch_time:", last_epoch_time, "this_epoch_time:", this_epoch_time, "offset:",
              offset, "start_idx:", start_idx, "duration_sec:", duration_sec)

        if offset < 0:
            offset += 86400
        offset -= duration_sec
        start_idx += offset * sampling_rate
        
        duration_sec = this_duratioin_sec
        max_duration_sec = duration_sec if duration_sec > max_duration_sec else max_duration_sec
        print('Current max duration:', max_duration_sec)
        
        time_duration[this_epoch_time] = duration_sec
        
        duration_epoch = duration_sec // 1
        # set indices for this epoch
        idx = int(start_idx) + np.arange(duration_sec * sampling_rate, dtype=np.int)
        start_idx += int(duration_sec * sampling_rate)
        if start_idx > len(raw_ch_df):
            break
        label_epoch = np.ones(duration_epoch, dtype=np.int) * label
#         print('label_epoch:', label_epoch, len(label_epoch), 'idx:', idx, len(idx),
#               "row['Time [hh:mm:ss]']", row['Time [hh:mm:ss]'],
#              "time_duration[this_epoch_time]", time_duration[this_epoch_time])
        
        ext_t = extend_time(row['Time [hh:mm:ss]'], time_duration[this_epoch_time])
                
        assert len(label_epoch) == len(idx)//int(sampling_rate) == len(ext_t)
        
        labels.append(label_epoch)
        label_idx.append(idx)
        time_stamps.append(ext_t)

        print("last_epoch_time:", last_epoch_time, "this_epoch_time:", this_epoch_time, "offset:", offset,
              "len(idx)", len(idx), "duration_sec:", duration_sec)
        
        last_epoch_time = this_epoch_time

    labels = np.hstack(labels)
    time_stamps = np.hstack(time_stamps)
    select_idx = np.arange(len(raw_ch_df))
    print("before intersect label: {}".format(select_idx.shape))
    label_idx = np.hstack(label_idx)
    print('labels.shape:', labels.shape, 'label_idx.shape:', label_idx.shape)
    select_idx = np.intersect1d(select_idx, label_idx)
    print("after intersect label: {}".format(select_idx.shape))

    # Remove movement and unknown stages if any
    raw_ch = raw_ch_df.values[select_idx]

    if len(raw_ch) % sampling_rate != 0:
        raise Exception("Something wrong")
    n_epochs = len(raw_ch) / sampling_rate
    print('n_epochs:', n_epochs)
    # Get epochs and their corresponding labels
    x = np.asarray(np.split(raw_ch, n_epochs)).astype(np.float32)
    y = labels.astype(np.int32)
    t = time_stamps.astype(str)

    print('len(x):', len(x), 'len(y):', len(y), 'len(t):', len(t))
    assert len(x) == len(y) == len(t)

    # Select on sleep periods
    print("Data after selection: {}, {}, {}".format(x.shape, y.shape, t.shape))
#     x = resample(x, 512*30, axis=1) if sampling_rate != 512 else x
    print('Before save {}, {}, {}'.format(x.shape, y.shape, t.shape))

    save_dict = {
        "x": x, 
        "y": y,
        "fs": sampling_rate,
        "t": t
    }
    np.savez(out_f, **save_dict)
    

Extracting EDF parameters from /share/wequ0318/CAP/physionet.org/files/capslpdb/1.0.0/n6.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 4047999  =      0.000 ... 31624.992 secs...
file /share/wequ0318/CAP/physionet.org/files/capslpdb/1.0.0/n6.edf contains channels ['LOC-A1', 'ROC-A2', 'EMG-EMG', 'C3-A2', 'O2-A1', 'ECG', 'TAG', 'THE']
selected channel O2-A1 and suffix is _O2_A1
output to /share/wequ0318/cap_clean/cap/n6_O2_A1.npz
Converting "time" to "<class 'numpy.int64'>"...
recording start time 2099-01-01 22:33:09, annotation start time 2099-01-01 22:56:19
Time difference is 1390.0 seconds
cnt: -1 last_epoch_time: 22:56:19 this_epoch_time: 22:56:19 offset: 0.0 start_idx: 177920.0 duration_sec: 0
Current max duration: 13
last_epoch_time: 22:56:19 this_epoch_time: 22:56:19 offset: 0.0 len(idx) 1664 duration_sec: 13
cnt: -2 last_epoch_time: 22:56:19 this_epoch_time: 22:57:08 offset: 49.0 start_idx: 179584.0 duration_sec: 13
Curren

cnt: -45 last_epoch_time: 23:21:32 this_epoch_time: 23:21:44 offset: 12.0 start_idx: 372352.0 duration_sec: 6
Current max duration: 16
last_epoch_time: 23:21:32 this_epoch_time: 23:21:44 offset: 6.0 len(idx) 512 duration_sec: 4
cnt: -46 last_epoch_time: 23:21:44 this_epoch_time: 23:21:56 offset: 12.0 start_idx: 373632.0 duration_sec: 4
Current max duration: 16
last_epoch_time: 23:21:44 this_epoch_time: 23:21:56 offset: 8.0 len(idx) 512 duration_sec: 4
cnt: -47 last_epoch_time: 23:21:56 this_epoch_time: 23:22:06 offset: 10.0 start_idx: 375168.0 duration_sec: 4
Current max duration: 16
last_epoch_time: 23:21:56 this_epoch_time: 23:22:06 offset: 6.0 len(idx) 896 duration_sec: 7
cnt: -48 last_epoch_time: 23:22:06 this_epoch_time: 23:22:35 offset: 29.0 start_idx: 376832.0 duration_sec: 7
Current max duration: 16
last_epoch_time: 23:22:06 this_epoch_time: 23:22:35 offset: 22.0 len(idx) 640 duration_sec: 5
cnt: -49 last_epoch_time: 23:22:35 this_epoch_time: 23:22:57 offset: 22.0 start_idx: 38

cnt: -85 last_epoch_time: 23:36:38 this_epoch_time: 23:36:59 offset: 21.0 start_idx: 488576.0 duration_sec: 8
Current max duration: 26
last_epoch_time: 23:36:38 this_epoch_time: 23:36:59 offset: 13.0 len(idx) 1024 duration_sec: 8
cnt: -86 last_epoch_time: 23:36:59 this_epoch_time: 23:37:41 offset: 42.0 start_idx: 491264.0 duration_sec: 8
Current max duration: 26
last_epoch_time: 23:36:59 this_epoch_time: 23:37:41 offset: 34.0 len(idx) 1280 duration_sec: 10
cnt: -87 last_epoch_time: 23:37:41 this_epoch_time: 23:39:35 offset: 114.0 start_idx: 496896.0 duration_sec: 10
Current max duration: 26
last_epoch_time: 23:37:41 this_epoch_time: 23:39:35 offset: 104.0 len(idx) 640 duration_sec: 5
cnt: -88 last_epoch_time: 23:39:35 this_epoch_time: 23:40:08 offset: 33.0 start_idx: 510848.0 duration_sec: 5
Current max duration: 26
last_epoch_time: 23:39:35 this_epoch_time: 23:40:08 offset: 28.0 len(idx) 768 duration_sec: 6
cnt: -89 last_epoch_time: 23:40:08 this_epoch_time: 23:40:33 offset: 25.0 star

last_epoch_time: 23:54:02 this_epoch_time: 23:54:11 offset: 3.0 len(idx) 1280 duration_sec: 10
cnt: -122 last_epoch_time: 23:54:11 this_epoch_time: 23:54:35 offset: 24.0 start_idx: 623616.0 duration_sec: 10
Current max duration: 26
last_epoch_time: 23:54:11 this_epoch_time: 23:54:35 offset: 14.0 len(idx) 768 duration_sec: 6
cnt: -123 last_epoch_time: 23:54:35 this_epoch_time: 23:55:31 offset: 56.0 start_idx: 626176.0 duration_sec: 6
Current max duration: 26
last_epoch_time: 23:54:35 this_epoch_time: 23:55:31 offset: 50.0 len(idx) 1152 duration_sec: 9
cnt: -124 last_epoch_time: 23:55:31 this_epoch_time: 23:55:54 offset: 23.0 start_idx: 633728.0 duration_sec: 9
Current max duration: 26
last_epoch_time: 23:55:31 this_epoch_time: 23:55:54 offset: 14.0 len(idx) 512 duration_sec: 4
cnt: -125 last_epoch_time: 23:55:54 this_epoch_time: 23:56:42 offset: 48.0 start_idx: 636032.0 duration_sec: 4
Current max duration: 26
last_epoch_time: 23:55:54 this_epoch_time: 23:56:42 offset: 44.0 len(idx) 640

cnt: -176 last_epoch_time: 00:39:54 this_epoch_time: 00:40:07 offset: 13.0 start_idx: 974848.0 duration_sec: 11
Current max duration: 26
last_epoch_time: 00:39:54 this_epoch_time: 00:40:07 offset: 2.0 len(idx) 896 duration_sec: 7
cnt: -177 last_epoch_time: 00:40:07 this_epoch_time: 00:40:44 offset: 37.0 start_idx: 976000.0 duration_sec: 7
Current max duration: 26
last_epoch_time: 00:40:07 this_epoch_time: 00:40:44 offset: 30.0 len(idx) 512 duration_sec: 4
cnt: -178 last_epoch_time: 00:40:44 this_epoch_time: 00:40:56 offset: 12.0 start_idx: 980352.0 duration_sec: 4
Current max duration: 26
last_epoch_time: 00:40:44 this_epoch_time: 00:40:56 offset: 8.0 len(idx) 1024 duration_sec: 8
cnt: -179 last_epoch_time: 00:40:56 this_epoch_time: 00:41:19 offset: 23.0 start_idx: 982400.0 duration_sec: 8
Current max duration: 26
last_epoch_time: 00:40:56 this_epoch_time: 00:41:19 offset: 15.0 len(idx) 640 duration_sec: 5
cnt: -180 last_epoch_time: 00:41:19 this_epoch_time: 00:41:57 offset: 38.0 start

...

last_epoch_time: 03:04:28 this_epoch_time: 03:04:58 offset: 20.0 len(idx) 3712 duration_sec: 29
cnt: -346 last_epoch_time: 03:04:58 this_epoch_time: 03:05:37 offset: 39.0 start_idx: 2091264.0 duration_sec: 29
Current max duration: 45
last_epoch_time: 03:04:58 this_epoch_time: 03:05:37 offset: 10.0 len(idx) 768 duration_sec: 6
cnt: -347 last_epoch_time: 03:05:37 this_epoch_time: 03:05:53 offset: 16.0 start_idx: 2093312.0 duration_sec: 6
Current max duration: 45
last_epoch_time: 03:05:37 this_epoch_time: 03:05:53 offset: 10.0 len(idx) 640 duration_sec: 5
cnt: -348 last_epoch_time: 03:05:53 this_epoch_time: 03:06:05 offset: 12.0 start_idx: 2095232.0 duration_sec: 5
Current max duration: 45
last_epoch_time: 03:05:53 this_epoch_time: 03:06:05 offset: 7.0 len(idx) 512 duration_sec: 4
cnt: -349 last_epoch_time: 03:06:05 this_epoch_time: 03:06:32 offset: 27.0 start_idx: 2096640.0 duration_sec: 4
Current max duration: 45
last_epoch_time: 03:06:05 this_epoch_time: 03:06:32 offset: 23.0 len(idx) 

last_epoch_time: 03:26:00 this_epoch_time: 03:26:34 offset: 28.0 len(idx) 640 duration_sec: 5
cnt: -382 last_epoch_time: 03:26:34 this_epoch_time: 03:26:45 offset: 11.0 start_idx: 2254080.0 duration_sec: 5
Current max duration: 45
last_epoch_time: 03:26:34 this_epoch_time: 03:26:45 offset: 6.0 len(idx) 1408 duration_sec: 11
cnt: -383 last_epoch_time: 03:26:45 this_epoch_time: 03:26:58 offset: 13.0 start_idx: 2256256.0 duration_sec: 11
Current max duration: 45
last_epoch_time: 03:26:45 this_epoch_time: 03:26:58 offset: 2.0 len(idx) 1536 duration_sec: 12
cnt: -384 last_epoch_time: 03:26:58 this_epoch_time: 03:28:35 offset: 97.0 start_idx: 2258048.0 duration_sec: 12
Current max duration: 45
last_epoch_time: 03:26:58 this_epoch_time: 03:28:35 offset: 85.0 len(idx) 1920 duration_sec: 15
cnt: -385 last_epoch_time: 03:28:35 this_epoch_time: 03:29:39 offset: 64.0 start_idx: 2270848.0 duration_sec: 15
Current max duration: 45
last_epoch_time: 03:28:35 this_epoch_time: 03:29:39 offset: 49.0 len(

cnt: -422 last_epoch_time: 04:53:42 this_epoch_time: 04:54:05 offset: 23.0 start_idx: 2924160.0 duration_sec: 12
Current max duration: 45
last_epoch_time: 04:53:42 this_epoch_time: 04:54:05 offset: 11.0 len(idx) 512 duration_sec: 4
cnt: -423 last_epoch_time: 04:54:05 this_epoch_time: 04:55:25 offset: 80.0 start_idx: 2926080.0 duration_sec: 4
Current max duration: 45
last_epoch_time: 04:54:05 this_epoch_time: 04:55:25 offset: 76.0 len(idx) 640 duration_sec: 5
cnt: -424 last_epoch_time: 04:55:25 this_epoch_time: 04:56:07 offset: 42.0 start_idx: 2936448.0 duration_sec: 5
Current max duration: 45
last_epoch_time: 04:55:25 this_epoch_time: 04:56:07 offset: 37.0 len(idx) 640 duration_sec: 5
cnt: -425 last_epoch_time: 04:56:07 this_epoch_time: 04:56:31 offset: 24.0 start_idx: 2941824.0 duration_sec: 5
Current max duration: 45
last_epoch_time: 04:56:07 this_epoch_time: 04:56:31 offset: 19.0 len(idx) 2816 duration_sec: 22
cnt: -426 last_epoch_time: 04:56:31 this_epoch_time: 04:57:13 offset: 42.

cnt: -461 last_epoch_time: 06:12:23 this_epoch_time: 06:12:36 offset: 13.0 start_idx: 3528064.0 duration_sec: 9
Current max duration: 45
last_epoch_time: 06:12:23 this_epoch_time: 06:12:36 offset: 4.0 len(idx) 384 duration_sec: 3
cnt: -462 last_epoch_time: 06:12:36 this_epoch_time: 06:13:58 offset: 82.0 start_idx: 3528960.0 duration_sec: 3
Current max duration: 45
last_epoch_time: 06:12:36 this_epoch_time: 06:13:58 offset: 79.0 len(idx) 768 duration_sec: 6
cnt: -463 last_epoch_time: 06:13:58 this_epoch_time: 06:14:37 offset: 39.0 start_idx: 3539840.0 duration_sec: 6
Current max duration: 45
last_epoch_time: 06:13:58 this_epoch_time: 06:14:37 offset: 33.0 len(idx) 384 duration_sec: 3
cnt: -464 last_epoch_time: 06:14:37 this_epoch_time: 06:16:54 offset: 137.0 start_idx: 3544448.0 duration_sec: 3
Current max duration: 45
last_epoch_time: 06:14:37 this_epoch_time: 06:16:54 offset: 134.0 len(idx) 384 duration_sec: 3
cnt: -465 last_epoch_time: 06:16:54 this_epoch_time: 06:17:28 offset: 34.0 

cnt: -504 last_epoch_time: 06:49:32 this_epoch_time: 06:50:06 offset: 34.0 start_idx: 3813632.0 duration_sec: 11
Current max duration: 45
last_epoch_time: 06:49:32 this_epoch_time: 06:50:06 offset: 23.0 len(idx) 896 duration_sec: 7
cnt: -505 last_epoch_time: 06:50:06 this_epoch_time: 06:50:18 offset: 12.0 start_idx: 3817472.0 duration_sec: 7
Current max duration: 45
last_epoch_time: 06:50:06 this_epoch_time: 06:50:18 offset: 5.0 len(idx) 384 duration_sec: 3
cnt: -506 last_epoch_time: 06:50:18 this_epoch_time: 06:51:04 offset: 46.0 start_idx: 3818496.0 duration_sec: 3
Current max duration: 45
last_epoch_time: 06:50:18 this_epoch_time: 06:51:04 offset: 43.0 len(idx) 1152 duration_sec: 9
cnt: -507 last_epoch_time: 06:51:04 this_epoch_time: 06:51:22 offset: 18.0 start_idx: 3825152.0 duration_sec: 9
Current max duration: 45
last_epoch_time: 06:51:04 this_epoch_time: 06:51:22 offset: 9.0 len(idx) 512 duration_sec: 4
cnt: -508 last_epoch_time: 06:51:22 this_epoch_time: 06:51:36 offset: 14.0 s